In [20]:
import os
# ptb 모듈 경로 추가
import sys
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/밑바닥부터시작하는딥러닝2')

In [21]:
import numpy as np
import collections
from common.functions import softmax, cross_entropy_error

from common.util import create_contexts_target,to_cpu, to_gpu
from common.optimizer import Adam
from common.trainer import Trainer

In [22]:
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/밑바닥부터시작하는딥러닝2/dataset')
from ptb import *

### Embedding 계층 구현   

In [23]:
class Embedding:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None
   
  # 순전파에서는 가중치 w의 특정 행을 추출 
  def forward(self, idx):
    W, = self.params
    self.idx = idx
    out = W[idx]     # idx에 해당하는 가중치 행렬의 행을 추출
    return out
  
  # 역전파에서는 가중치 기울기 dW를 꺼낸다음, dW의 원소를 0으로 설정한 후, idx에 해당하는 행에 dout을 더한다.-> idx의 원소가 중복될 수 있으므로 더해야함. 
  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    np.add.at(dW, self.idx, dout)  # dW의 특정 행에 dout을 더함
    return None

### Embedding dot 계층 구현  

In [24]:
class EmbeddingDot:
  def __init__(self, W):
    self.embed = Embedding(W)
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None                   # 순전파 시의 계산 결과를 담아두는 변수

  def forward(self, h, idx):            # h는 은닉층 뉴런의 출력, idx는 단어 ID 배열
    target_W = self.embed.forward(idx)  # Embedding 계층의 forward 메서드 호출, idx에 해당하는 가중치 행렬의 행을 추출, 여기서 idx는 정답 레이블의 단어 ID이다. 
    out = np.sum(target_W * h, axis=1)  # target_W와 h의 행렬 곱 계산

    self.cache = (h, target_W)        
    return out

  def backward(self, dout):
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0], 1)

    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

### 네거티브 샘플링 구현  

In [25]:
# 단어 빈도 수를 구해 확률분포에 따라 샘플링하는 함수

words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words, size=2, replace=False)  # words에서 무작위로 5개 선택, 중복 없음

# 확률분포에 따라 샘플링
p = [0.1, 0.05, 0.5, 0.2, 0.05, 0.1] # 합이 1인 확률분포
np.random.choice(words, p=p)  # words에서 확률분포 p에 따라 샘플링

'.'

In [26]:
# 네거티브 샘플링 구현
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [27]:
# GPU 사용 여부 설정
GPU = False

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size) # UnigramSampler 클래스의 인스턴스 생성
target = np.array([1, 3, 0]) # 3개의 데이터를 미니배치로 묶어 처리한다고 가정, 긍정적 예
negative_sample = sampler.get_negative_sample(target) # 부정적 예 샘플링
print(negative_sample)

[[2 3]
 [1 2]
 [2 3]]


In [28]:
class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None  # sigmoid의 출력
        self.t = None  # 정답 데이터

    def forward(self, x, t):
        self.t = t
        self.y = 1 / (1 + np.exp(-x))

        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = (self.y - self.t) * dout / batch_size
        return dx

In [29]:
# 네거티브 샘플링을 수행하는 손실함수 구현
# NegativeSampling 클래스 내부에는 EmbeddingDot 계층과 SigmoidWithLoss 계층이 포함되어 있다.
class NegativeSamplingLoss:
  def __init__(self, W, corpus, power=0.75, sample_size=5):
    self.sample_size = sample_size                                          # 샘플링 수
    self.sampler = UnigramSampler(corpus, power, sample_size)               # UnigramSampler 클래스의 인스턴스 생성
    self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]  # 긍정적 예 1개 + 부정적 예 sample_size개, loss_layers[0]는 긍정적 예에 대한 손실을 구하는 계층이다.
    self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] # 긍정적 예 1개 + 부정적 예 sample_size개, embed_dot_layers[0]는 긍정적 예에 대한 점수를 구하는 계층이다.
    
    self.params, self.grads = [], []                                        # 모든 가중치와 기울기를 리스트에 모은다.
    for layer in self.embed_dot_layers:
      self.params += layer.params
      self.grads += layer.grads
  
  # 순전파, 긍정적 예와 부정적 예의 점수를 구하고, 손실을 계산한다.
  def forward(self, h, target):
    batch_size = target.shape[0]
    negative_sample = self.sampler.get_negative_sample(target)  # 부정적 예 샘플링, UnigramSampler 클래스의 get_negative_sample 메서드 호출
    
    # 긍정적 예의 순전파
    score = self.embed_dot_layers[0].forward(h, target)          # 긍정적 예의 점수, EmbeddingDot 클래스의 forward 메서드 호출, h와 target 간의 내적을 계산. 
    correct_label = np.ones(batch_size, dtype=np.int32)          # 긍정적 예의 레이블은 1
    loss = self.loss_layers[0].forward(score, correct_label)      # 긍정적 예의 손실, SigmoidWithLoss 클래스의 forward 메서드 호출, score를 SigmoidWithLoss 계층에 입력하여 손실을 계산.
    
    # 부정적 예의 순전파
    negative_label = np.zeros(batch_size, dtype=np.int32)        # 부정적 예의 레이블은 0
    for i in range(self.sample_size):
      negative_target = negative_sample[:, i]                    # 부정적 예의 타깃
      score = self.embed_dot_layers[1 + i].forward(h, negative_target)  # 부정적 예의 점수, EmbeddingDot 클래스의 forward 메서드 호출, h와 negative_target 간의 내적을 계산.
      loss += self.loss_layers[1 + i].forward(score, negative_label)    # 부정적 예의 손실, SigmoidWithLoss 클래스의 forward 메서드 호출, score를 SigmoidWithLoss 계층에 입력하여 손실을 계산.
      
    return loss
  
  # 역전파
  def backward(self, dout=1):
    dh = 0
    for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
      dscore = l0.backward(dout)  # SigmoidWithLoss 계층의 역전파, SigmoidWithLoss 클래스의 backward 메서드 호출
      dh += l1.backward(dscore)   # EmbeddingDot 계층의 역전파, EmbeddingDot 클래스의 backward 메서드 호출
    return dh

### SimpleCBOW 모델을 개선한 CBOW 모델, Embedding 계층과 Negative Sampling Loss 계층을 적용했다.  

In [30]:
class CBOW:
  def __init__(self, vocab_size, hidden_size, window_size, corpus): # voab_size:어휘수, hidden_size:은닉층 뉴런 수, window_size:맥락의 크기, corpus:말뭉치
    V, H = vocab_size, hidden_size                  

    # 가중치 초기화
    W_in = 0.01 * np.random.randn(V, H).astype('f')
    W_out = 0.01 * np.random.randn(V, H).astype('f')

    # 계층 생성
    self.in_layers = []
    for i in range(2 * window_size):
      layer = Embedding(W_in)              # Embedding 계층 사용, 2*window_size만큼 생성, 예) window_size가 1이면 2개의 Embedding 계층 생성
      self.in_layers.append(layer)         # Embedding 계층을 인스턴스 변수 in_layers에 배열로 저장
    self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)  # NegativeSamplingLoss 계층을 인스턴스 변수 ns_loss에 저장

    # 모든 가중치와 기울기를 리스트에 모은다.
    layers = self.in_layers + [self.ns_loss]  # in_layers와 ns_loss를 합쳐 layers에 저장, 모든 계층을 리스트에 모은다.
    self.params, self.grads = [], []
    for layer in layers:
      self.params += layer.params             # params에 각 계층의 가중치를 저장
      self.grads += layer.grads               # grads에 각 계층의 기울기를 저장

    # 인스턴스 변수에 단어의 분산 표현을 저장한다.
    self.word_vecs = W_in   # 인스턴스 변수 word_vecs에 W_in을 저장해 단어의 분산 표현에 접근할 수 있게 한다. 사실상 word_vecs를 구하기 위해 필요한 것은 W_in이다.
    
  # 순전파
  def forward(self, contexts, target):     # contexts는 맥락, target은 타깃을 나타내는데 이 두 인수는 단어 ID의 배열이다.
    h = 0
    for i, layer in enumerate(self.in_layers):
      h += layer.forward(contexts[:, i])  # Embedding 계층의 forward 메서드 호출, contexts[:, i]는 각 맥락의 단어 ID를 추출
    h *= 1 / len(self.in_layers)           # 은닉층 노드 h의 값이 맥락의 수만큼 나눠서 평균값을 구한다. 

    loss = self.ns_loss.forward(h, target) # NegativeSamplingLoss 계층의 forward 메서드 호출, h와 target을 입력하여 손실을 계산
    return loss
  
  # 역전파
  def backward(self, dout=1):
    dout = self.ns_loss.backward(dout)  # NegativeSamplingLoss 계층의 역전파, NegativeSamplingLoss 클래스의 backward 메서드 호출
    dout *= 1 / len(self.in_layers)     # 은닉층 노드 h의 기울기를 맥락의 수만큼 나눠서 평균값을 구한다.

    for layer in self.in_layers:
      layer.backward(dout)              # Embedding 계층의 역전파, Embedding 클래스의 backward 메서드 호출

### CBOW 모델 학습 코드  

In [31]:
# 하이퍼 파라미터 설정
window_size = 1
hidden_size = 5
batch_size = 3 # 미니배치 크기
max_epoch = 2

# 데이터 읽기
corpus, word_to_id, id_to_word = load_data('train')
vocab_size = len(word_to_id)
vocab_size
contexts, target = create_contexts_target(corpus, window_size) # 맥락과 타깃 생성하는 함수 호출
if GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
    
# 모델 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()



| 에폭 1 |  반복 1 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 41 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 61 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 81 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 101 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 121 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 141 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 161 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 181 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 201 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 221 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 241 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 261 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 281 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 301 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 321 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 341 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 361 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 381 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 401 / 309862 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 421 / 309862 | 시간 0[s] | 손실

KeyboardInterrupt: 

In [18]:
import pickle

In [19]:
# 학습이 끝난 후 단어의 분산 표현을 추출
word_vecs = model.word_vecs
if GPU:
    word_vecs = to_cpu(word_vecs)

# 단어 별 분산 표현을 사전에 저장
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)